In [1]:
#my reference: https://pypi.org/project/conllu/

In [2]:
pip install conllu==3.1.1

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install conll-df

Note: you may need to restart the kernel to use updated packages.


In [7]:
from io import open
from conllu import parse_tree_incr
from conllu import parse_incr
from conllu import parse
from collections import OrderedDict
import pandas as pd
import itertools
import re

In [13]:
file = 'TamilTB.v0.1\data\TamilTB.v0.1.utf8.conll'
data_file = open(file, "r", encoding="utf-8")


In [14]:
#Setting precision of dataframe as 0
pd.set_option('precision', 0)

#Setting Dataframe display to max
pd.set_option('display.max_rows', None)

In [15]:
#Converting conll to a dataframe
columns = ['id', 'form', 'lemma', 'upos', 'xpos', 'feats', 'head', 'Morpheme']
df = pd.DataFrame(columns = columns)
for tokenlist in parse_incr(data_file):
    for word in tokenlist:
        df = df.append({'id': word["id"], 
                        'form' : word["form"],
                        'lemma' : word["lemma"],
                        'upos' : word["upos"],
                        'xpos' : word["xpos"],
                        'feats' : word["feats"],
                        'head' : word["head"]},
                        ignore_index = True
                        )
df.head(10)

,id,form,lemma,upos,xpos,feats,head,Morpheme
0,1,சென்னை,சென்னை,N,NEN-3SN--,"{'Cas': 'N', 'Per': '3', 'Num': 'S', 'Gen': 'N'}",2,NaN
1,2,அருகே,அருகே,P,PP-------,None,18,NaN
2,3,ஸ்ரீ,ஸ்ரீ,N,NEN-3SN--,"{'Cas': 'N', 'Per': '3', 'Num': 'S', 'Gen': 'N'}",4,NaN
3,4,பெரும்புதூரில்,பெரும்புதூர்,N,NEL-3SN--,"{'Cas': 'L', 'Per': '3', 'Num': 'S', 'Gen': 'N'}",18,NaN
4,5,கிரீன்,கிரீன்,N,NEN-3SN--,"{'Cas': 'N', 'Per': '3', 'Num': 'S', 'Gen': 'N'}",6,NaN
5,6,பீல்டு,பீல்டு,N,NEN-3SN--,"{'Cas': 'N', 'Per': '3', 'Num': 'S', 'Gen': 'N'}",11,NaN
6,7,(,(,Z,Z:-------,None,6,NaN
7,8,நவீன,நவீனம்,J,JJ-------,None,6,NaN
8,9,),),Z,Z:-------,None,6,NaN
9,10,விமான,விமானம்,N,NO--3SN--,"{'Per': '3', 'Num': 'S', 'Gen': 'N'}",11,NaN


In [16]:
#Function to extract the morphemes by stripping away the lemma
def morpheme_extract(lemmalist, wordlist):
    if len(lemmalist)<len(wordlist):
        for i in range(len(lemmalist)):
            if (wordlist[i] != lemmalist[i]):
                difference.append(wordlist[i])
        index = i
        for i in range(len(wordlist)):
            if i > index:
                difference.append(wordlist[i])
                morpheme = ''.join(difference)
                row["Morpheme"] = morpheme
    else:
        row["Morpheme"] = ""


In [17]:
# Going through each row in the dataframe and calling the function to strip away the morphemes
for index, row in df.iterrows():
    word = row["form"]
    wordlist = [ch for ch in word] 
    lemma = row["lemma"]
    lemmalist = [ch for ch in lemma] 
    difference = []
    morpheme_extract(lemmalist, wordlist)
    difference[:5]

In [10]:
df.head()

,id,form,lemma,upos,xpos,feats,head,Morpheme
0,1,சென்னை,சென்னை,N,NEN-3SN--,"{'Cas': 'N', 'Per': '3', 'Num': 'S', 'Gen': 'N'}",2,
1,2,அருகே,அருகே,P,PP-------,None,18,
2,3,ஸ்ரீ,ஸ்ரீ,N,NEN-3SN--,"{'Cas': 'N', 'Per': '3', 'Num': 'S', 'Gen': 'N'}",4,
3,4,பெரும்புதூரில்,பெரும்புதூர்,N,NEL-3SN--,"{'Cas': 'L', 'Per': '3', 'Num': 'S', 'Gen': 'N'}",18,ில்
4,5,கிரீன்,கிரீன்,N,NEN-3SN--,"{'Cas': 'N', 'Per': '3', 'Num': 'S', 'Gen': 'N'}",6,


In [20]:
#Dropping id column
df.drop(columns='id', inplace=True)
df.head()

,form,lemma,upos,xpos,feats,head,Morpheme
0,சென்னை,சென்னை,N,NEN-3SN--,"{'Cas': 'N', 'Per': '3', 'Num': 'S', 'Gen': 'N'}",2,
1,அருகே,அருகே,P,PP-------,None,18,
2,ஸ்ரீ,ஸ்ரீ,N,NEN-3SN--,"{'Cas': 'N', 'Per': '3', 'Num': 'S', 'Gen': 'N'}",4,
3,பெரும்புதூரில்,பெரும்புதூர்,N,NEL-3SN--,"{'Cas': 'L', 'Per': '3', 'Num': 'S', 'Gen': 'N'}",18,ில்
4,கிரீன்,கிரீன்,N,NEN-3SN--,"{'Cas': 'N', 'Per': '3', 'Num': 'S', 'Gen': 'N'}",6,


In [21]:
#Reading sentences csv 
file_path = "Sentence_index.csv"
sentence_df = pd.read_csv(file_path, encoding="utf-8")
sentence_df.dropna(inplace=True)
sentence_df.drop(columns='id', inplace=True)
sentence_df.reset_index(inplace=True)
sentence_df.head()

,index,NoSpaceAfter
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0


In [22]:
#Adding NoSpaceAfter column to dataframe. The 'no_space_after' will be set to 1 if the following token is part of the current token.
#Whenever the splitting takes place this attribute will be set to 1 for the first token.
#For example, The 'no_space_after' attribute for pATukAkkap  will be 1. Whereas the 'no_space_after' attribute for um will be 0.  
df['NoSpaceAfter'] = sentence_df['NoSpaceAfter']
df.head()

,form,lemma,upos,xpos,feats,head,Morpheme,NoSpaceAfter
0,சென்னை,சென்னை,N,NEN-3SN--,"{'Cas': 'N', 'Per': '3', 'Num': 'S', 'Gen': 'N'}",2,,0
1,அருகே,அருகே,P,PP-------,None,18,,0
2,ஸ்ரீ,ஸ்ரீ,N,NEN-3SN--,"{'Cas': 'N', 'Per': '3', 'Num': 'S', 'Gen': 'N'}",4,,0
3,பெரும்புதூரில்,பெரும்புதூர்,N,NEL-3SN--,"{'Cas': 'L', 'Per': '3', 'Num': 'S', 'Gen': 'N'}",18,ில்,0
4,கிரீன்,கிரீன்,N,NEN-3SN--,"{'Cas': 'N', 'Per': '3', 'Num': 'S', 'Gen': 'N'}",6,,0


In [23]:
#Adding postpositions and clitics to morpheme list
for i in range(0, len(df)):
    if df.loc[i,'NoSpaceAfter']==1:
        df.loc[i+1, 'Morpheme'] = df.loc[i+1, 'form']
        df.loc[i+1, 'NoSpaceAfter'] = 2


In [24]:
#Removing everything expect nouns and verbs
for index, row in df.iterrows():
    if (row["upos"] != 'N') and (row["upos"] != 'V') and (row['NoSpaceAfter'] != 2):
        df.drop(index, inplace = True)

        

In [26]:
#Creating dataframe out of the frequencies of the morphemes
value_counts_df = pd.DataFrame(df["Morpheme"].value_counts())
value_counts_df.reset_index(inplace=True)
value_counts_df = value_counts_df.rename(columns = {'index':'Morpheme', 'Morpheme':'Counts'})
value_counts_df["Counts"][0]=0
value_counts_df.head()

C:\Users\sonia\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,Morpheme,Counts
0,,0
1,கள்,210
2,உம்,209
3,யில்,155
4,ஆக,127


In [27]:
#Merging value counts with the original dataframe
merged_df = pd.merge(df, value_counts_df, on="Morpheme", how = "left")
merged_df.head()

,form,lemma,upos,xpos,feats,head,Morpheme,NoSpaceAfter,Counts
0,சென்னை,சென்னை,N,NEN-3SN--,"{'Cas': 'N', 'Per': '3', 'Num': 'S', 'Gen': 'N'}",2,,0,0
1,ஸ்ரீ,ஸ்ரீ,N,NEN-3SN--,"{'Cas': 'N', 'Per': '3', 'Num': 'S', 'Gen': 'N'}",4,,0,0
2,பெரும்புதூரில்,பெரும்புதூர்,N,NEL-3SN--,"{'Cas': 'L', 'Per': '3', 'Num': 'S', 'Gen': 'N'}",18,ில்,0,111
3,கிரீன்,கிரீன்,N,NEN-3SN--,"{'Cas': 'N', 'Per': '3', 'Num': 'S', 'Gen': 'N'}",6,,0,0
4,பீல்டு,பீல்டு,N,NEN-3SN--,"{'Cas': 'N', 'Per': '3', 'Num': 'S', 'Gen': 'N'}",11,,0,0


In [28]:
# Removing all the blank morpheme rows
for index, row in merged_df.iterrows():
    if row["Morpheme"] == '':
        merged_df.drop(index, inplace = True)


In [29]:
#Cleaning up to only get the noun and verb morphemes along with their value counts
columns = ['Morpheme', 'Counts','upos', 'xpos', 'NoSpaceAfter']
adjusted_morpheme_df = pd.DataFrame(columns = columns)
adjusted_morpheme_df["Morpheme"] = merged_df["Morpheme"]
adjusted_morpheme_df["Counts"] = merged_df["Counts"]
adjusted_morpheme_df["upos"] = merged_df["upos"]
adjusted_morpheme_df["xpos"] = merged_df["xpos"]
adjusted_morpheme_df["NoSpaceAfter"] = merged_df["NoSpaceAfter"]
adjusted_morpheme_df = adjusted_morpheme_df.drop_duplicates(subset = ['Morpheme', 'NoSpaceAfter', 'Counts'])
adjusted_morpheme_df = adjusted_morpheme_df.sort_values(by='Counts', ascending=False)
adjusted_morpheme_df.head()

,Morpheme,Counts,upos,xpos,NoSpaceAfter
130,கள்,210,N,NNN-3PA--,1
172,கள்,210,N,NNN-3PN--,0
30,உம்,209,T,Tv-------,2
10,யில்,155,N,NNL-3SN--,0
653,யில்,155,N,NNL-3SN--,1


In [30]:
# Making postpositions/clitics morpheme list
clitic_df = adjusted_morpheme_df[adjusted_morpheme_df['NoSpaceAfter']==2]
clitic_df.head()

,Morpheme,Counts,upos,xpos,NoSpaceAfter
30,உம்,209,T,Tv-------,2
131,ஆக,127,P,PP-------,2
7,ஆன,61,T,Tg-------,2
358,பட்ட்,52,V,VT-T---PA,2
235,உள்ளது,47,V,VR-T3SNAA,2


# Replacing Morpheme list with Character

In [58]:
original_df = clitic_df[['Morpheme']]
original_df.count()

Morpheme    62
dtype: int64

In [59]:
original_df

,Morpheme
30,உம்
131,ஆக
7,ஆன
358,பட்ட்
235,உள்ளது
16,உள்ளார்
127,இலிருந்து
385,உடன்
349,பட்டது
415,இல்லை


In [67]:
orig_array = original_df['Morpheme'].values
orig_array

array(['உம்', 'ஆக', 'ஆன', 'பட்ட்', 'உள்ளது', 'உள்ளார்', 'இலிருந்து',
       'உடன்', 'பட்டது', 'இல்லை', 'ஆகவ்', 'பட்டு', 'உள்ள', 'உள்ளனர்',
       'குறித்து', 'ஏ', 'ஏயே', 'ஆவது', 'ப்பட', 'ஓ', 'ஆகத்', 'உள்',
       'இருந்தார்', 'ஓடு', 'இருக்கிறது', 'ஆம்', 'உள்ளத்', 'ஒட்டி',
       'உள்ளன', 'பட்ட', 'தவிர', 'இருந்தால்', 'இருக்கும்', 'இருந்தது',
       'இருந்த', 'அடுத்து', 'உள்ளேன்', 'ஆகச்', 'கூட', 'படி', 'ஆகப்',
       'பட்டார்', 'ஆகக்', 'பட', 'ஆனால்', 'இருந்து', 'எல்லாம்', 'உரிய',
       'இருக்கிறீர்கள்', 'பகுதிகளில்', 'உள்ளதால்', 'சட்டத்தின்',
       'இருக்கினறனர்', 'அன்று', 'உள்ளோம்', 'உள்ளாகவே', 'படக்', 'ஆகியதால்',
       'நிலையில்', 'இருப்பத்', 'பகுதியில்', 'விட'], dtype=object)

In [72]:
import numpy as np
len(orig_array)

62

In [75]:
type(orig_array[0])

str

In [98]:
new_array = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']
len(new_array)
new_array[0]

'A'

In [79]:
sentence = 'ஆக பட்ட்'

In [104]:
list = []
list.append(sentence)
list.append(sentence)
list

['ஆக பட்ட்', 'ஆக பட்ட்']

In [80]:
for char in sentence:
    print(char);

ஆ
க
 
ப
ட
்
ட
்


In [91]:
sentence2 = 'ஆக'
for i in range(len(orig_array)):
    if sentence2 == orig_array[i]:
        sentence2 = new_array[i];

In [92]:
sentence2

'B'

In [106]:
i = 0;
sentence_list = [];
for char in sentence:
    sentence_list.append(char);
    if char == ' ':
        i = i+1;

In [107]:
sentence_list

['ஆ', 'க', ' ', 'ப', 'ட', '்', 'ட', '்']

In [113]:
new = np.array(5)
i=0
new[0] = 'hi';
new
# for char in sentence:
#     new[i] = char;
#     if char == ' ':
#         i = i+1;

IndexError: too many indices for array

In [118]:
# Wingdings
from tkinter import Tk, font
root = Tk()
#font.families()

In [134]:
#print('\u2700')
print('\u2326')

⌦


In [161]:
'\u2B04'

'⬄'

In [162]:
wingdings = ['\u21E6','\u21E7','\u21E8','\u21E9','\u21F3','\u237D','\u2303','\u237D','\u2303','\u2316','\u2324','\u2325',
             '\u2326','\u2327', '\u232B','\u232D','\u238B','\u23F3','\u2423','\u24C5','\u24FF','\u261C','\u261E','\u261D','\u261F','\u25A0','\u25A1','\u2620','\u262F','\u2639','\u263A','\u263C','\u267F','\u26F7','\u2701','\u2702','\u2706','\u2707','\u2708','\u270C','\u270D','\u2744','\u2B00','\u2B01','\u2B02','\u2B03','\u2B04','\u2318','\u2327','\u25C6','\u2605','\u2726','\u2734','\u2735','\u2736','\u2739','\u2756','\u2BD0','\u2BD1','\u272A','\u2730','\u2714']
             
#'\u1F4DA','\u1F378','\u1F393','\u1F3A7','\u1F3AC','\u1F3AD','\u1F3C2','\u1F3C4','\u1F3C6','\u1F3CA','\u1F408','\u1F415','\u1F41F','\u1F426']
wingdings
#len(wingdings)

['⇦',
 '⇧',
 '⇨',
 '⇩',
 '⇳',
 '⍽',
 '⌃',
 '⍽',
 '⌃',
 '⌖',
 '⌤',
 '⌥',
 '⌦',
 '⌧',
 '⌫',
 '⌭',
 '⎋',
 '⏳',
 '␣',
 'Ⓟ',
 '⓿',
 '☜',
 '☞',
 '☝',
 '☟',
 '■',
 '□',
 '☠',
 '☯',
 '☹',
 '☺',
 '☼',
 '♿',
 '⛷',
 '✁',
 '✂',
 '✆',
 '✇',
 '✈',
 '✌',
 '✍',
 '❄',
 '⬀',
 '⬁',
 '⬂',
 '⬃',
 '⬄',
 '⌘',
 '⌧',
 '◆',
 '★',
 '✦',
 '✴',
 '✵',
 '✶',
 '✹',
 '❖',
 '⯐',
 '⯑',
 '✪',
 '✰',
 '✔']

In [141]:
sentence2 = 'ஆக'
for i in range(len(orig_array)):
    if sentence2 == orig_array[i]:
        sentence2 = wingdings[i];
sentence2

'⌧'

In [20]:
#Word list excluding postpostions
root_df = adjusted_morpheme_df[adjusted_morpheme_df['NoSpaceAfter']<2]
root_df.reset_index(inplace=True)
root_df.head()

,index,Morpheme,Counts,upos,xpos,NoSpaceAfter
0,130,கள்,210,N,NNN-3PA--,1
1,172,கள்,210,N,NNN-3PN--,0
2,10,யில்,155,N,NNL-3SN--,0
3,653,யில்,155,N,NNL-3SN--,1
4,2,ில்,111,N,NEL-3SN--,0


In [21]:
#Creating morpheme list with no duplicates (without postpositions)
no_repeat_df = root_df.drop_duplicates(subset = ['Morpheme'])


# Getting Morpheme list for each type of noun and verb

## Noun - Accusative Case

In [33]:
# Creating function to make morphlist with counts
def morphcount(input_df):
    for i in range(len(input_df)):
        word = input_df.loc[i, 'Morpheme']
        a = [ch for ch in word]
        b= []
        n = len(a)
        for num_splits in range(n):
            for splits in itertools.combinations(range(1, n), num_splits):
                splices = zip([0] + list(splits), list(splits) + [n])
                b.append([a[i:j] for i, j in splices])

        df = pd.DataFrame.from_records(b)
        if i == 0:
            morphs = df[i]
        for j in range(len(df.columns)):
            morphs = morphs.append(df[j])
        
        morphs = morphs.map(lambda x: ''.join(x),na_action='ignore')
        morphs.dropna(inplace=True)


In [37]:
NA_df = no_repeat_df[no_repeat_df['xpos'].str.contains(r'(NNA.*)')]
NA_df.reset_index(inplace=True)
NA_df.drop_duplicates(inplace=True)
NA_df

C:\Users\keetu\anaconda3\lib\site-packages\pandas\core\strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)
<ipython-input-37-f466f7b98591>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  NA_df.drop_duplicates(inplace=True)


,level_0,index,Morpheme,Counts,upos,xpos,NoSpaceAfter
0,18,390,களை,46,N,NNA-3PN--,0
1,70,28,ஙகளை,14,N,NNA-3PN--,0
2,83,249,ததைய்,11,N,NNA-3SN--,1
3,87,1011,களைய்,11,N,NNA-3PA--,1
4,114,92,களைப்,7,N,NNA-3PN--,0
5,132,2640,ைக்,5,N,NNA-3SN--,0
6,133,1659,களைக்,5,N,NNA-3PN--,0
7,143,108,ஙகளைப்,4,N,NNA-3PN--,0
8,147,1228,ைப்,4,N,NNA-3SN--,0
9,148,1236,யைத்,4,N,NNA-3SN--,0


In [35]:
morphcount(NA_df)
morphs_df = pd.DataFrame(morphs)
columns = ['values']
morphs_values_df = pd.DataFrame(morphs_df.value_counts(), columns = columns)
morphs_values_df.sort_values(by=['values'], ascending=False).head()

,values
0,
்,998
இ,576
வ,548
ை,431
த,383


## Noun - Dative Case

In [38]:
ND_df = no_repeat_df[no_repeat_df['xpos'].str.contains(r'(N.D.*)')]
ND_df.reset_index(inplace=True)
ND_df.drop_duplicates(inplace=True)
ND_df

C:\Users\keetu\anaconda3\lib\site-packages\pandas\core\strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)
<ipython-input-38-260405493b89>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ND_df.drop_duplicates(inplace=True)


,level_0,index,Morpheme,Counts,upos,xpos,NoSpaceAfter
0,15,506,க்கு,49,N,NND-3SN--,0
1,57,169,ுக்கு,18,N,NPDF3PH-A,0
2,64,1281,களுக்கு,16,N,NND-3PN--,0
3,68,258,வுக்கு,14,N,NED-3SN--,0
4,69,948,களுக்க்,14,N,NND-3PN--,1
5,72,1760,க்க்,14,N,NED-3SN--,1
6,84,70,ததுக்கு,11,N,NND-3SN--,0
7,96,1740,ுக்க்,9,N,NND-3SN--,1
8,122,2376,ததுக்க்,6,N,NND-3SN--,1
9,131,1294,ஙகளுக்கு,5,N,NND-3PN--,0


In [41]:
morphcount(ND_df)
morphs_df = pd.DataFrame(morphs)
columns = ['values']
morphs_values_df = pd.DataFrame(morphs_df.value_counts(), columns = columns)
morphs_values_df.sort_values(by=['values'], ascending=False).head()

,values
0,
்,998
இ,576
வ,548
ை,431
த,383


In [ ]:
##Next steps
## make a list based on what you see for each tense (noun - stem+plural+case  verb - stem+tense+PNG)
##Read the grammar textbook again make sure theres nothing missing